## 📸 OpenCV를 이용한 비디오 처리

### 비디오 읽고 쓰기
#### 비디오 읽기

In [3]:
import cv2

In [4]:
cam = cv2.VideoCapture("./video.avi")
#cam.set(cv2.CAP_PROP_POS_FRAMES,1800)
while cam.isOpened():
    ret, frame = cam.read()
    cv2.imshow("frame",frame)
    if cv2.waitKey(30) & 0xFF == ord("q"):
        break
cam.release()
cv2.destroyAllWindows()

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\highgui\src\window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


#### 비디오 쓰기

In [5]:
cam = cv2.VideoCapture(0)
h = int(cam.get(3))
w = int(cam.get(4))

fourcc = cv2.VideoWriter_fourcc(*"DIVX")
video_write = cv2.VideoWriter("saved_out.avi", fourcc, 25.0, (w,h))

while cam.isOpened():
    ret, frame = cam.read()
    video_write.write(frame)
    cv2.imshow("video",frame)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cam.release()
video_write.release()
cv2.destroyAllWindows()

### 비디오에 대한 기본 연산
#### Grayscale 변환

In [1]:
import cv2

In [2]:
cam = cv2.VideoCapture("./video.avi")
while cam.isOpened():
    ret, frame = cam.read()
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow("gray_frame", gray_frame)
    cv2.imshow("original", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cv2.destroyAllWindows()

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


#### 색상 추적

In [1]:
import cv2
import numpy as np

In [3]:
def detect(img):
    lower_range = np.array([40,150,150],dtype="uint8")
    upper_range = np.array([70,255,255],dtype="uint8")
    img = cv2.inRange(img, lower_range, upper_range)
    cv2.imshow("Range",img)
    m = cv2.moments(img)
    if m["m00"] != 0:
        x = int(m["m10"]/m["m00"])
        y = int(m["m01"]/m["m00"])
    else:
        x = 0
        y = 0
    return (x,y)

In [7]:
cam = cv2.VideoCapture("./video.avi")
last_x = 0
last_y = 0

In [8]:
while cam.isOpened():
    ret, frame = cam.read()
    cur_x, cur_y = detect(frame)
    cv2.line(frame,(cur_x, cur_y),(last_x, last_y),(0,0,200),5)
    last_x = cur_x
    last_y = cur_y
    cv2.imshow("frame",frame)
    if cv2.waitKey(1) &0xFF == ord("q"):
        break
        
cam.release()
cv2.destroyAllWindows()

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\core\src\arithm.cpp:1743: error: (-215:Assertion failed) ! _src.empty() in function 'cv::inRange'


### 물체 추적
#### Kernelized Correlation Filter

In [5]:
import cv2

In [7]:
print(cv2.__version__)

4.4.0


In [4]:
tracker = cv2.TrackerKCF_create()   ##Opencv 3.4 이상 3버전에서만 가능
cam = cv2.VideoCapture("./video.mp4")
for i in range(5):
    ret, frame = cam.read()
obj = cv2.selectROI("Tracking",frame)
ok = tracker.init(frame, obj)

AttributeError: module 'cv2.cv2' has no attribute 'TrackerKCF_create'

In [ ]:
while True:
    ret, frame = cam.read()
    upd, obj = tracker.update(frame)
    if upd:
        x1 = (int(obj[0]), int(obj[1]))
        x2 = (int(obj[0] + obj[2]), int(obj[1] + obj[3]))
        cv2.rectangle(frame,x1,x2,(255,0,0))
    cv2.imshow("Track object",frame)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
cam.release()
cv2.detroyAllWindows()

#### Luckas Kanade Tracker

In [8]:
import numpy as np
import cv2

In [10]:
cap = cv2.VideoCapture("./video.avi")
feature_params = dict(maxCorners = 1000, 
                      qualityLevel = 0.3, 
                      minDistance = 7, 
                      blockSize = 5, 
                      useHarrisDetector = 1,
                      k=0.04)
lk_params = dict(winSize=(15,15),maxLevel = 2)
color = np.random.randint(0,255,(1000,3))
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

In [11]:
mask = np.zeros_like(old_frame)
count = 0

In [17]:
while cap.isOpened():
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask,(a,b),(c,d),color[i].tolist(),2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)
    cv2.imshow("frame",img)
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break
    old_gray = frame_gray.copy()
    count = count + 1
    if count % 100 == 0:
        p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
    else:
        p0 = good_new.reshape(-1,1,2)
cv2.destroyAllWindows()
cap.release()

<ipython-input-17-b94d6021f6c2>:11: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  mask = cv2.line(mask,(a,b),(c,d),color[i].tolist(),2)
<ipython-input-17-b94d6021f6c2>:12: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
